# set constants

In [1]:
import numpy as np
from scipy.optimize import fsolve

# Given OIS rates (annualized percentage, converted to decimal)
ois_rates = {
    .5: 0.0025, 1: 0.0030, 2: 0.00325, 3: 0.00335,
    4: 0.0035, 5: 0.0036, 7: 0.0040, 10: 0.0045,
    15: 0.0050, 20: 0.00525, 30: 0.0055
}

# instantiate dictionary for DF and f
dict_D = {}
dict_f = {}

def D_1(f):
    """
    function to return Discount factor for 0.5 year only, f is zero rate for that 1 year
    """
    return 1/(1+f/360)**180

def D_n(f):
    """
    function to return Discount factor for 1 year only,  f is zero rate for that 1 year
    """
    return 1/(1+f/360)**360

# Get the 0.5 and 1 year DF and f rates

In [2]:
f_0 = 360*((.5*ois_rates[.5]+1)**(1/180)-1)
D_0 = D_1(f_0)

Solve for 1 year f and D(0, 1y) by using a fsolver.
1. make function creator for PV fixed
2. make function creator for PV flt
3. put two together in another function
4. use combined function in fsolver

In [3]:
tenor = 1

# state fxns for PV_flt, PV_fix
fn_fixed_1y = lambda f_1: ois_rates[tenor]/((1+f_0/360)**(180)*(1+f_1/360)**(180))
fn_float_1y = lambda  f_1: 1 - 1/((1 + f_0 / 360)**(180) * (1 + f_1 / 360)**(180))
eqn = lambda f_1: fn_fixed_1y(f_1) - fn_float_1y(f_1)

# use solver for PV_flt-PV_fix=0
res = fsolve(eqn, 1e-5)[0]
res, fn_fixed_1y(res)-fn_float_1y(res)

(0.0034925961006445987, -1.6675896774565047e-14)

In [4]:
# Save results to dictionaries
dict_f[tenor] = res
dict_D[tenor] = D_0*D_1(res)

In [5]:
dict_f, dict_D

({1: 0.0034925961006445987}, {1: 0.9970089730807411})

# Now compute for 2 year

$$
PV_{fix}^{2y OIS} = [D_0(0,1y) + D_0(0,2y)] \times R_{2y OIS}
$$

we can consider the first term as the sum of all known discount factors and the unknown discount factor

In [6]:
def fn_pv_fix(f_n, tenor):
    """
    Return function for PV of flt leg parameterised by zero rate of the last year. 
    only works from year 2 onwards. Dictionaries are taken as workbook constants
    """
    sum_df = sum(dict_D.values())
    return (sum_df + dict_D[tenor-1]*D_n(f_n)) * ois_rates[tenor]

$$
PV_{flt}^{2y OIS} = 
D_o(0,1y) \times \left[ \left( 1 + \frac{f_0}{360} \right)^{180} \left( 1 + \frac{f_1}{360} \right)^{180} - 1 \right] + D_o(0,2y) \times \left[ \left( 1 + \frac{f_2}{360} \right)^{360} - 1 \right]

$$

In [7]:
def fn_pv_flt(f_n,tenor):
    """
    Return function for PV of flt leg parameterised by zero rate of the last year. 
    only works from year 2 onwards. Dictionaries are taken as workbook constants
    """
    return 1-dict_D[tenor-1]*D_n(f_n)

In [8]:
def eqn(f_n, t):
    # print("tenor",tenor)
    return fn_pv_fix(f_n,t) - fn_pv_flt(f_n,t)

for tenor in range(2,6):
    f = fsolve(eqn,.003, tenor)[0]

    dict_f[tenor] = f
    dict_D[tenor] = dict_D[tenor-1]*D_n(f)

dict_D, dict_f

({1: 0.9970089730807411,
  2: 0.9935307459132384,
  3: 0.9900151412182555,
  4: 0.9861166497152494,
  5: 0.9821841197332517},
 {1: 0.0034925961006445987,
  2: 0.0034947783831644763,
  3: 0.003544788862661668,
  4: 0.00394560518903897,
  5: 0.003995890364441599})

In [ ]:
def eqn3(df, rate, tenor):
    sum1 = sum(dict_D.values())
    sum2 = (7-5+1) * (dict_D[tenor-1]*D_n(f_n)+dict_D[tenor-1])/2 - dict_D[tenor-1]
    return rate*(sum1+sum2) - (1-dict_D[tenor-1]*D_n(f_n))

In [19]:
f = fsolve(eqn3, .003, args=(ois_rates[7],6))
f

array([0.01000574])

In [17]:
dict_D[tenor-1]*D_n(f)

array([0.97629915])

In [23]:
d1 = .95
d3 = .99
d2 = (d1+d3)/2

1/d2

1.0309278350515465

In [26]:
r1 = 1/d1-1
r2 = (1/d2)**.5-1
r3 = (1/d3)**(1/3)-1

r1, r2, r3

(0.05263157894736836, 0.015346165133619083, 0.003355729847985822)

In [27]:
(r1+r3)/2

0.027993654397677092

# cannot interpolate zero rates even if we can interpolate discount factors